In [2]:
import pandas as pd
import json

In [3]:
sdf = pd.read_csv('merge_csv/source.csv')
# sdf = sdf.drop('rule_type', axis=1)
# sdf.to_csv('./source.csv')

cid_df = pd.read_csv('merge_csv/cid.csv', index_col='cid')

def get_obj_type(obj):
    basicTypes = obj.basicTypes
    if isinstance(basicTypes, str):
        basicTypes = eval(basicTypes)
    _type = None
    for x in basicTypes:
        _type = x['enBasicType']
        if _type in {'Procedure', 'Medication'}:
            break
    obj['cid_type'] = _type
    return obj

cid_df = cid_df.apply(get_obj_type, axis=1)

cid1_df = cid_df[['cnDesc','cid_type']].add_suffix('1')
cid2_df = cid_df[['cnDesc','cid_type']].add_suffix('2')

sdf = pd.merge(sdf, cid1_df, left_on='cid1', right_index=True)
sdf = pd.merge(sdf, cid2_df, left_on='cid2', right_index=True)

In [273]:
sdf[sdf.cid_type1.isna() | sdf.cid_type2.isna()]

,enabled,deleted,cr_time,up_time,comment,id,cid1,rel,cid2,rela,source,is_source,cnDesc1,cid_type1,cnDesc2,cid_type2
1027,1,1,2022-07-18 17:33:51,2022-11-03 18:16:35,fangkankan 2022-11-03 update deleted from [0] ...,95244181,SO00016893,RO,SO00612908,operation_has_contraindication_with_patient_co...,SRT,999,妊娠首三个月,None,阿达木单抗,Medication
1434,1,0,2022-10-28 16:47:33,2022-10-28 16:47:33,chen.chong. 2022-10-28 bulk insert con relatio...,95277691,SO00016893,RO,SO00158698,operation_has_contraindication_with_patient_co...,SRT,SRT,妊娠首三个月,None,惠菲宁,Medication
2463,1,1,2022-07-18 17:33:47,2022-11-03 18:16:36,fangkankan 2022-11-03 update deleted from [0] ...,95243591,SO00612908,RO,SO00016893,patient_condition_is_contraindication_of_opera...,SRT,999,阿达木单抗,Medication,妊娠首三个月,None
2898,1,0,2022-10-28 16:47:30,2022-10-28 16:47:30,chen.chong. 2022-10-28 bulk insert con relatio...,95277647,SO00158698,RO,SO00016893,patient_condition_is_contraindication_of_opera...,SRT,SRT,惠菲宁,Medication,妊娠首三个月,None


In [275]:
sdf['cid_type2'].value_counts()

DiseaseDisorder    1126
Medication          894
Procedure           359
BrandName           257
SignSymptom          90
Qualifier            56
Exam                 51
ProcedureDevice      24
Finding              21
AgeGroup             13
Anatomy              13
Treatment             3
Standard              3
VitalSign             2
Time                  2
Behavior              2
Course                1
Name: cid_type2, dtype: int64

In [249]:
def check_rule_type(obj):
    if obj['cid_type2'] == 'Procedure':
        _type = "治疗禁忌"
    elif obj['cid_type2'] == 'Medication':
        _type = "药品禁忌"
    else:
        _type = "其他"
    return _type

rule_type = sdf.apply(check_rule_type, axis=1)

rule_type.name = 'rule_type'

In [250]:
sdf = sdf.join(rule_type)

sdf[sdf.rule_type == '药品禁忌']

,enabled,deleted,cr_time,up_time,comment,id,cid1,rel,cid2,rela,source,is_source,cnDesc1,cid_type1,cnDesc2,cid_type2,rule_type
13,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242227,SO02517047,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,眼科手术,Procedure,泰毕全,Medication,药品禁忌
27,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242241,SO01375826,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,脊髓手术,Procedure,泰毕全,Medication,药品禁忌
31,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242245,SO00676193,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,脑部治疗,Procedure,泰毕全,Medication,药品禁忌
38,1,0,2022-07-07 15:08:50,2022-07-07 15:08:50,fangkankan 2022-07-07 bulk insert con relation...,95243015,SO01536350,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,泰毕全,Medication,泰毕全,Medication,药品禁忌
1503,1,0,2022-07-07 15:08:49,2022-07-07 15:08:49,fangkankan 2022-07-07 bulk insert con relation...,95243005,SO01536350,RO,SO01536350,operation_is_contraindication_of_operation,SRT,SRT,泰毕全,Medication,泰毕全,Medication,药品禁忌
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,1,0,2022-07-18 17:33:47,2022-07-18 17:33:47,fangkankan 2022-07-18 bulk insert con relation...,95243860,SO00407654,RO,SO00038783,patient_condition_is_contraindication_of_opera...,SRT,SRT,吗丁啉,Medication,伊曲康唑,Medication,药品禁忌
1540,1,0,2022-07-18 17:26:22,2022-07-18 17:26:22,fangkankan 2022-07-18 bulk insert con relation...,95243281,SO00407703,RO,SO00013411,operation_is_contraindication_of_operation,SRT,SRT,得理多,Medication,单胺氧化酶抑制剂,Medication,药品禁忌
1551,1,0,2022-07-18 17:26:22,2022-07-18 17:26:22,fangkankan 2022-07-18 bulk insert con relation...,95243292,SO02193205,RO,SO02529319,operation_is_contraindication_of_operation,SRT,SRT,重酒石酸去甲肾上腺素注射液,Medication,儿茶酚胺类药,Medication,药品禁忌
2738,1,0,2022-07-18 17:33:47,2022-07-18 17:33:47,fangkankan 2022-07-18 bulk insert con relation...,95243866,SO02193205,RO,SO02529318,patient_condition_is_contraindication_of_opera...,SRT,SRT,重酒石酸去甲肾上腺素注射液,Medication,含卤素的麻醉剂,Medication,药品禁忌


In [251]:
def is_visible(obj, pool:set):
    if obj.cid_type2 == 'Medication' and obj.cid_type1 == 'Medication':
        key = tuple(sorted([obj.cid1, obj.cid2]))
        if key in pool:
            return False
        else:
            pool.add(key)
            return True
    return True

In [252]:
pool=set()
visible = sdf.apply(is_visible, axis=1, pool=pool)
visible.name = 'visible'

In [253]:
sdf = sdf.join(visible)

In [254]:
sdf[(sdf.visible & (sdf.rule_type == '药品禁忌'))]

,enabled,deleted,cr_time,up_time,comment,id,cid1,rel,cid2,rela,source,is_source,cnDesc1,cid_type1,cnDesc2,cid_type2,rule_type,visible
13,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242227,SO02517047,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,眼科手术,Procedure,泰毕全,Medication,药品禁忌,True
27,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242241,SO01375826,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,脊髓手术,Procedure,泰毕全,Medication,药品禁忌,True
31,1,0,2022-07-07 14:10:11,2022-07-07 14:10:11,fangkankan 2022-07-07 bulk insert con relation...,95242245,SO00676193,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,脑部治疗,Procedure,泰毕全,Medication,药品禁忌,True
38,1,0,2022-07-07 15:08:50,2022-07-07 15:08:50,fangkankan 2022-07-07 bulk insert con relation...,95243015,SO01536350,RO,SO01536350,operation_has_contraindication_with_operation,SRT,SRT,泰毕全,Medication,泰毕全,Medication,药品禁忌,True
352,1,0,2022-07-07 14:10:29,2022-07-13 12:14:43,fangkankan 2022-07-07 bulk insert con relation...,95242637,SO00012149,RO,SO01536350,operation_has_contraindication_with_patient_co...,SRT,SRT,肝脏疾病,DiseaseDisorder,泰毕全,Medication,药品禁忌,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,1,0,2022-07-07 14:10:09,2022-07-07 14:10:09,fangkankan 2022-07-07 bulk insert con relation...,95242210,SO01569096,RO,SO01190862,operation_is_contraindication_of_operation,SRT,SRT,拜瑞妥,BrandName,达比加群,Medication,药品禁忌,True
1492,1,0,2022-07-07 14:10:09,2022-07-07 14:10:09,fangkankan 2022-07-07 bulk insert con relation...,95242211,SO01569096,RO,SO02529080,operation_is_contraindication_of_operation,SRT,SRT,拜瑞妥,BrandName,口服抗凝剂,Medication,药品禁忌,True
1496,1,0,2022-07-07 14:10:09,2022-07-07 14:10:09,fangkankan 2022-07-07 bulk insert con relation...,95242215,SO01569096,RO,SO02529078,operation_is_contraindication_of_operation,SRT,SRT,拜瑞妥,BrandName,其他抗凝剂,Medication,药品禁忌,True
1498,1,0,2022-07-07 14:10:09,2022-07-07 14:10:09,fangkankan 2022-07-07 bulk insert con relation...,95242217,SO01569096,RO,SO01432664,operation_is_contraindication_of_operation,SRT,SRT,拜瑞妥,BrandName,普通肝素,Medication,药品禁忌,True
